In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob,os
import sys
import scipy
from importlib import  reload
from time import process_time 
#from libraries.lib_gather_data import get_hhid_FIES
from datetime import datetime
%load_ext autoreload

In [2]:
#from shock_libraries import *
#from plotting_libraries import *
#from response_libraries import get_response_sp
#
from income_shock_libraries_ps import *
#
#from libraries.lib_country_dir import set_directories, load_survey_data, get_places_dict
#from libraries.lib_get_hh_savings import get_hh_savings
#from libraries.pandas_helper import broadcast_simple

In [3]:
# formatting & aesthetics
font = {'family':'sans serif', 'size':10}
plt.rc('font', **font)
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.facecolor'] = 'white'
sns.set_style("white")

sns_pal = sns.color_palette('Set1', n_colors=8, desat=.4)
greys_pal = sns.color_palette('Greys', n_colors=9)

In [4]:
mr = merge_rank(rank_file='./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv', labor_file='./2015FIES/LFSJul2015_merge.csv',outfile='./temp/_labor_rank_merge_test.csv',merge_col='a09_pqkb')
mr

/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv


,Unnamed: 0,prov,mun,bgy,ea,shsn,hcn,cc101_lno,creg,stratum,...,cempst1,newempstat,j12c09_grade,j12c11_gradtech,j12c11course,pwgt,LFS_sector,w_home,demand_scale,E_sector
0,0,Abra,1,8,0,1,8,1,Cordillera Administrative Region,717,...,Employed,1.0,614,no,NaN,198.3980,eduhealth,3.0,0.5,CSRP services
1,1,Abra,1,8,0,1,8,2,Cordillera Administrative Region,717,...,Employed,1.0,Second Year College,no,NaN,224.0499,ag,2.0,1.0,Crop Farming and Gardening
2,2,Abra,1,8,0,1,8,3,Cordillera Administrative Region,717,...,Not in the labor force,3.0,First Year College,no,NaN,234.2929,NaN,NaN,NaN,NaN
3,3,Abra,1,8,0,1,8,4,Cordillera Administrative Region,717,...,Not in the labor force,3.0,Elementary Graduate,no,NaN,150.6083,NaN,NaN,NaN,NaN
4,4,Abra,1,8,0,2,19,1,Cordillera Administrative Region,717,...,Employed,1.0,684,no,NaN,228.2236,transportation,1.0,0.0,"Transportation, Storage and Comm. Services"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206509,206509,Cotabato City,4,35,0,4,83,9,Region XII - SOCCSKSARGEN,487,...,Employed,1.0,Elementary Graduate,no,NaN,351.3405,ag,2.0,0.0,Forestry and Hunting
206510,206510,Cotabato City,4,35,0,5,110,1,Region XII - SOCCSKSARGEN,487,...,Not in the labor force,3.0,No Grade Completed,no,NaN,249.6065,NaN,NaN,NaN,NaN
206511,206511,Cotabato City,4,35,0,5,110,2,Region XII - SOCCSKSARGEN,487,...,Employed,1.0,No Grade Completed,no,NaN,238.6860,retail,0.0,1.0,Wholesale and Retail
206512,206512,Cotabato City,4,35,0,5,110,3,Region XII - SOCCSKSARGEN,487,...,Employed,1.0,Second Year High School,no,NaN,483.5432,ag,2.0,1.0,Crop Farming and Gardening


In [5]:
# a = rand_weighted_shock_3dim_v2_edit()
# a

In [9]:
#def entre_shock(s_sector='LFS_sector'):
"""
"""
s_sector = 'LFS_sector'
mr = merge_rank(rank_file='./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv', labor_file='./2015FIES/LFSJul2015_merge.csv',outfile='./temp/_labor_rank_merge_test.csv',merge_col='a09_pqkb')
if not 'LFS_sector' in mr.columns:
    mr = mr.rename(columns={'LFS_sector_x': 'LFS_sector'})
    # get subset: a09_pqkb

if 'hhid_lfs' in mr.columns:
    mr_subset = mr[['hhid_lfs','cc101_lno','LFS_sector','a09_pqkb','c19_pclass','demand_scale', 'w_home','E_sector','pwgt']]
else:
    mr_subset = mr[['cc101_lno','LFS_sector','a09_pqkb','c19_pclass','demand_scale', 'w_home','E_sector','pwgt']]
    
    
mr_subset
indexNames = mr_subset[mr_subset['a09_pqkb'] == 'nan' ].index
# Delete these row indexes from dataFrame
mr_subset.drop(indexNames , inplace=True)
mr_subset = mr_subset.reset_index(drop=True)
# get subset: c19_pclass

indexNames2 = mr_subset[mr_subset['c19_pclass'] == 'nan' ].index

    # Delete these row indexes from dataFrame
mr_subset.drop(indexNames2 , inplace=True)
mr_subset = mr_subset.reset_index(drop=True)
mr_subset

# make new column of combined string a09 && c19:
mr_subset['a09c19'] = mr_subset['a09_pqkb'] +'-'+mr_subset['c19_pclass']

    # enforce string:
mr_subset['a09_pqkb'] = [str(q).strip() for q in mr_subset['a09_pqkb']] # enforce type = string
mr_subset['LFS_sector'] = [str(q).strip() for q in mr_subset['LFS_sector']] # enforce type = string
mr_subset['c19_pclass'] = [str(q).strip() for q in mr_subset['c19_pclass']] # enforce type = string
mr_subset['a09c19'] = [str(q).strip() for q in mr_subset['a09c19']] # enforce type = string
if 'E_sector' in mr_subset.columns:
    mr_subset['E_sector'] = [str(q).strip() for q in mr_subset['E_sector']] # enforce type = string

mr_subset


x = np.unique(mr_subset.c19_pclass)
x

### create entrepreneurial table:
#rslt_df = dataframe[dataframe['Percentage'] > 80] 
#subsetDataFrame = dfObj[dfObj['Product'].isin(['Mangos', 'Grapes']) ]


# or nonag_wage : private household, private establishment, govt corporation, with pay (family owned business)
df_nonag = mr_subset #[~mr_subset['c19_pclass'].isin(['Self Employed', 'Employer','Without Pay (Family owned Business)'])]
df_nonag


# generate fraction by  ENTREPRENEURIAL sector
#df_nonag['desc_count'] = df_nonag.groupby('a09_pqkb')['a09_pqkb'].transform('count')# count unique jobs and append to mr_subset
#df_nonag['sector_count'] = df_nonag.groupby('E_sector')['E_sector'].transform('count') #count total unique sectors and append to mr_subset

df_nonag['sector_count'] = df_nonag.groupby(s_sector)['pwgt'].transform('sum') #count total unique sectors and append to mr_subset
df_nonag['desc_count'] = df_nonag.groupby('a09_pqkb')['pwgt'].transform('sum') #count total unique sectors and append to mr_subset

df_nonag['sector_frac'] = df_nonag['desc_count'] / df_nonag['sector_count'] # get fraction of sector as weightin

#### for now we will leave non-ag here

# for entrepreneurial income: self employed, employer, withOUT pay (family owned business)
df_ent = mr_subset #[~mr_subset['c19_pclass'].isin(['Self Employed', 'Employer','Without Pay (Family owned Business)'])]

#####  ###### #####
#RUNNING AG
df_ent = df_nonag

##### ###### #####

#####
# here, need to insert a new column that merges a09 and c19 -- done
# then, drop duplicates off of this column, so that we can minimize computation

# still need logic to build the logic for each job sector
## may need to restructure this whole section of code

#####
    # drop duplicates (now that overall weighting established)
df_ent = df_ent.drop_duplicates(subset='a09_pqkb')
df_ent = df_ent.reset_index(drop=True)



    # generate probability and combine with relative weighting
df_ent['partial_prob'] = np.nan
df_ent['third_col'] = np.nan
df_ent['dummy'] = np.nan

    # incorporate Kayenat tables into 'di' &&
    # nested logic to incorporate 0-4 scale for social distancing measures
    ## where scores of 0 & 1 result in complete job lost, due to unable to distance
i=0
while i < len(df_ent):

    if df_ent.demand_scale[i] == 0:

        # incorporate 0-4 scale logic:

        if df_ent.w_home[i] == 0:
            df_ent.partial_prob[i] = 0

        elif df_ent.w_home[i] == 1:
            df_ent.partial_prob[i] = 0

        else:
            df_ent.partial_prob[i] = df_ent.sector_frac[i] * (random.randint(0,50)/100)


    elif df_ent.demand_scale[i] == 0.5: 

        # incorporate 0-4 scale logic:
        if df_ent.w_home[i] == 0:
            df_ent.partial_prob[i] = 0

        elif df_ent.w_home[i] == 1:
            df_ent.partial_prob[i] = 0

        else: 
            df_ent.partial_prob[i] = df_ent.sector_frac[i] * (random.randint(50,100)/100)

    elif df_ent.demand_scale[i] == 1.0:
        df_ent.partial_prob[i] = df_ent.sector_frac[i]
    else:
        df_ent.dummy[i] = -99
    

# incorporate 3rd column modifiers here:
    if (df_ent['c19_pclass'][i] == "Gov't/Gov't Corporation"):
        df_ent.partial_prob[i] = 1  # essentially reverts the random uniform logic implemented above

        
    i = i + 1 
    # remove nans in summing fields, and dummy storage
del df_ent['dummy']




# get mean probability by sector:


#storage['fa'] = np.nan
storage = pd.DataFrame(columns=['fa', 'di'], index=[np.unique(df_ent[s_sector])])
storage.index.names = ['sector']
# # save to separate var for testing    
# rand_weighted_shock = storage

# rand_weighted_shock
# #return(rand_weighted_shock)

# storage = pd.DataFrame(columns=['fa', 'di'], index=[np.unique(df_ent.E_sector)])

# iterate through loop of sector names
for seclist in np.unique(df_ent[s_sector]):
    pillow = 1 - (df_ent[df_ent[s_sector] == seclist].partial_prob.sum()) # KEY: logic inverted here-- now a probability of 1- change job loss = 'fraction affected'

    storage.loc[seclist,'fa'] = pillow
    storage.loc[seclist,'di'] = 1
    
    print('seclist ='+ seclist)


tstamp = (datetime.now().strftime("%Y%m%d_%H%M"))
storage.to_csv('./temp/table_full_'+s_sector+'_' +tstamp+'.csv')
#storage
#    return(storage)

The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
seclist =ag
seclist =construction
seclist =eduhealth
seclist =extraction
seclist =finance
seclist =food_entertainment
seclist =government
seclist =information
seclist =manufacturing
seclist =other
seclist =professional_services
seclist =retail
seclist =transportation
seclist =utilities
seclist =wholesale


/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/lib/python3.6/site

In [7]:
 np.unique(df_ent[s_sector])

array(['ag', 'construction', 'eduhealth', 'extraction', 'finance',
       'food_entertainment', 'government', 'information', 'manufacturing',
       'other', 'professional_services', 'retail', 'transportation',
       'utilities', 'wholesale'], dtype=object)

In [10]:
xx = entre_shock()
xx
 


NameError: name 'entre_shock' is not defined

In [ ]:
def generate_shock_100_entre():  # initialize shock sector storage dataframe
   
    '''
   current hard coding for sensitivity analysis, 20200413: requires cleaning for further implementation
   - addition of modularity
   - 
   - current functionality:
       - outputs csv to location: './temp/sect_iter_100.csv
       - containing data frame with 101 simulations of <rand_weighted_shock_distance():
    - runtime: ~10minutes
   '''

    stor = entre_shock()
    del stor['di']

    # set number of iterations
    p = 0
    n_iter = 99

    # model and store stochastic sector response
    while p < n_iter:
        
        new_val = entre_shock()
        del new_val['di']
        new_val = new_val.rename(columns={'fa': ('iter'+str(p))})

        # pd.merge(labor,rank, on=merge_col, how='left')
        stor = pd.merge(stor,new_val,on='sector', how='left')
        p = p+ 1
        print(p)
        
    tstamp = (datetime.now().strftime("%Y%m%d_%H%M"))
    stor.to_csv('./temp/entre_shock_mc_fullsector_'+tstamp+ '.csv')


In [ ]:
generate_shock_100_entre()

In [ ]:
def get_shock_stats_entre():
    # generate shock table statistics
    #df['mean'] = df.mean(axis=1)

    # load csv to dataframe:
    #dfs = pd.read_csv('./temp/sect_iter_100.csv') # original
    dfs = pd.read_csv('./temp/entre_shock_mc_fullsector_20200427_1252.csv') # modified 20200420
    # set index to LFS_sector
    dfs.set_index('sector')

    # compute statistics:
    dfs['mean'] = dfs.mean(axis=1)
    #print(dfs['mean'])
    dfs['std_dev'] = dfs.std(axis=1)
    #print(dfs['std_dev'])

    #round to 3 dec:
    dfs['mean'] = [(round(q, 3)) for q in dfs['mean']]
    dfs['std_dev'] = [(round(q, 3)) for q in dfs['std_dev']]

    # new datafame storing just info:
    df_stat = dfs[['sector','mean','std_dev']].set_index('sector')
    df_stat
    # df_stat.to_csv('./temp/phi_get_shock_input.csv') # original
    tstamp = (datetime.now().strftime("%Y%m%d_%H%M"))
    df_stat.to_csv('./temp/phi_fullsector_'+tstamp+'.csv') # modified 20200420
    
    
    
    
    return(df_stat)


In [ ]:
get_shock_stats_entre()

In [ ]:
## create the shock table
def get_phi_shock(flavor=0):
    """
    20200426: loads shock tables
    
    input: flavor
        options:
            0 -- default shock table, in original model
            1 -- sectoral shock table, based on scoring
            2 -- nonag_shock table, sector names v2
            3 -- entrepreneurial shock table, sector names v2
    
    
    """
    if flavor == 0:
        # default shock table:
        shock_default = { 'ag':           [  0,  0],
                 'mining':        [  0,  0],
                 'utilities':     [  0,  0],
                 'construction':  [0.5,1.0],
                 'manufacturing': [0.1,1.0],
                 'wholesale':     [0.1,1.0],
                 'retail':        [0.5,1.0],
                 'transportation':[0.5,1.0],
                 'information':   [0.1,1.0],
                 'finance':       [0.1,1.0],
                 'professional_services':[0.1,1.0],
                 'eduhealth':     [0.1,1.0],
                 'food_entertainment':[0.8,1.0],
                 'government':    [  0,  0],
                 'other':         [0.8,1.0]}
        df_shock = pd.DataFrame(data=shock_default).T
        df_shock.columns = ['fa','di']
        df_shock.index.name = 'sector'
        shock_table = df_shock
    
    if flavor == 1:
        # sectoral shock table
        df = pd.read_csv('./temp/phi_shocks/phi_shock_3dimv2.csv').set_index('LFS_sector')
        df = df.rename(columns={'mean': 'fa', 'std_dev':'di'}) 
        df['di'] = 1
        
        #print(flavor)
        #print(df)
        shock_table = df
        
    if flavor == 2:
        #nonag shock table
        df = pd.read_csv('./temp/phi_shocks/phi_nonag.csv').set_index('sector')
        df = df.rename(columns={'mean': 'fa', 'std_dev':'di'}) 
        df['di'] = 1
       # print(flavor)
        shock_table = df
        
    if flavor == 3:
        #entrepreneurial shock table
        df = pd.read_csv('./temp/phi_shocks/phi_entre.csv').set_index('sector')
        df = df.rename(columns={'mean': 'fa', 'std_dev':'di'}) 
        df['di'] = 1
        shock_table = df
        
        
        #print(flavor)
   
    
    return(shock_table)

In [ ]:
xl = get_phi_shock(3)
xl